# Product Pricing Informaion Gathering with Python

This Project will use an existing product list with part number to gather updated pricing by scraping the manufacturer's site and gathering the updated info into a new CSV file.

#### 1. Importing dependancies, adjusting settings and creating the backup function.

In [2]:
import pandas as pd 
import numpy as np
import requests
import time
import random
from bs4 import BeautifulSoup as BS
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from requests import get
import re
import pickle
pd.set_option('display.max_rows', 500)

In [3]:
#Setup backup functions to create binary backups of variables in case of a kernal crash or need to stop and restart the kernal:
def save(filename, *args):
    # Get global dictionary
    glob = globals()
    d = {}
    for v in args:
        # Copy over desired values
        d[v] = glob[v]
    with open(filename, 'wb') as f:
        # Put them in the file 
        pickle.dump(d, f)

def load(filename):
    # Get global dictionary
    glob = globals()
    with open(filename, 'rb') as f:
        for k, v in pickle.load(f).items():
            # Set each global variable to the value from the file
            glob[k] = v

def delay() -> None:
    time.sleep(random.uniform(5, 10))
    return None            

#### 2. Import CRM parts file with the part numbers needed.  I've manually removed information that's not needed so that we just have the part numbers for products we are interested in finding. With this particular CRM the Ergotron part numbers start with ERG allowing for filtering out other manufacturers within the program but in this example case I've done that already.

In [5]:
pn2 = []
pn2 = pd.read_csv('CRM.csv',  thousands=",")
pn2['Product Name'] = pn2['Product Name'].astype(str)
pn2 = pn2.loc[pn2['Product Name'].str.startswith('ERG', na=False)]
print(pn2.shape)
pn2.head()


(326, 4)


,Product Name,Description,Part Number,Price
0,ERG-33-397-085,"WorkFit-T, Sit-Stand Desktop Workstation (black)",33-397-085,$449.00
1,ERG-61-367-030,CareFit Enclosure (snow white) Wall-mount comp...,61-367-030,"$1,233.00"
2,ERG-45-478-026,HX Wall Mount Monitor Arm (Polished Aluminum),45-478-026,$325.00
3,ERG-24-313-026,"WorkFit-A, Single LD",24-313-026,$409.00
4,ERG-60-610-062,"StyleView Sit-Stand Enclosure (bright white), ...",60-610-062,"$1,112.00"


#### 3. Making some modifications to the dataframe to help with data manipulation later.

In [6]:
pn2['Part Number'] = pn2['Product Name'].str.lstrip('ERG-')
pn2['PN_len'] = pn2['Part Number'].apply(len)
pn2.loc[pn2['PN_len'] == 10, 'Part Short'] = pn2['Part Number'].str[:6]
pn2.loc[pn2['PN_len'] == 11, 'Part Short'] = pn2['Part Number'].str[:9]
pn2.loc[pn2['Part Number'].str.contains('C50'), 'Part Short'] = pn2['Part Number'].str[:-2]
pn2['Part Short'] = pn2['Part Short'].fillna(pn2['Part Number'])
pn2

,Product Name,Description,Part Number,Price,PN_len,Part Short
0,ERG-33-397-085,"WorkFit-T, Sit-Stand Desktop Workstation (black)",33-397-085,$449.00,10,33-397
1,ERG-61-367-030,CareFit Enclosure (snow white) Wall-mount comp...,61-367-030,"$1,233.00",10,61-367
2,ERG-45-478-026,HX Wall Mount Monitor Arm (Polished Aluminum),45-478-026,$325.00,10,45-478
3,ERG-24-313-026,"WorkFit-A, Single LD",24-313-026,$409.00,10,24-313
4,ERG-60-610-062,"StyleView Sit-Stand Enclosure (bright white), ...",60-610-062,"$1,112.00",10,60-610
5,ERG-98-017,"LX Arm Grommet Mount for WorkFit-T, WorkFit-TL...",98-017,$17.00,6,98-017
6,ERG-97-617,WorkFit Convert-to-LCD & Laptop Kit from Dual ...,97-617,$62.00,6,97-617
7,ERG-47-058-200,"Pivot Assembly, Single Pivot with P/L and 1.5""...",47-058-200,$93.00,10,47-058
8,ERG-20-076-100,"Command Post kiosk system 3""x3""x80"" steel post...",20-076-100,$220.00,10,20-076
9,ERG-24-383-026,"WorkFit-P- Sit-Stand Workstation platform, 15 ...",24-383-026,$265.00,10,24-383


#### 4. Collect the Unique Item IDs Used by The Traget Site
Due to the structure of the target wesite it was determined that the most effective way to extract the info needed was to first identify the product ID codes used in the website to pull up the pricing information from a json file. So this will be step 1 of a 2 step scraping process where we will get all the product IDs for the items in our starting dataframe and and add them to a list.

In [9]:
item =[]
itme2 = []
def delay() -> None:
    time.sleep(random.uniform(5, 10))
    return None
item = pn2['Part Short']
item2 = pn2['Part Number']
headers = {'User-Agent': 'Mozilla/5.0'}
prod_id_df = []
part_num = []
p_id = []
actual_pn = []

for part, part2 in zip(item, item2):
    item_code = part
    full_prt = part2
    url = "https://partner.ergotron.com/en-us/products/product-details/{}#" .format(item_code)
    delay()
    r = requests.get(url, headers=headers)
    soup = BS(r.text, 'html.parser')
    try:
        target = soup.find("div", {"id": "dnn_ctr443_ContentPane"})
        target = str(target)
        tar = target.split("productId =",1)[1]
        tar = tar.split(",",1)[0]
        tar = re.sub("[^0-9]", "", tar)
    except:
        try:
            item_code = part2
            full_prt = part2
            url = "https://partner.ergotron.com/en-us/products/product-details/{}#" .format(item_code)
            delay()
            r = requests.get(url, headers=headers)
            soup = BS(r.text, 'html.parser')
            target = soup.find("div", {"id": "dnn_ctr443_ContentPane"})
            target = str(target)
            tar = target.split("productId =",1)[1]
            tar = tar.split(",",1)[0]
            tar = re.sub("[^0-9]", "", tar)
        except:    
            try:
                item_code = part2 
                full_prt = part2
                url = "https://partner.ergotron.com/en-us/products/product-details/{}" .format(item_code)
                delay()
                r = requests.get(url, headers=headers)
                soup = BS(r.text, 'html.parser')
                target = soup.find("div", {"id": "dnn_ctr443_ContentPane"})
                target = str(target)
                tar = target.split("productId =",1)[1]
                tar = tar.split(",",1)[0]
                tar = re.sub("[^0-9]", "", tar)
            except:    
                tar = "URL nonmatch"
    p_id.append(tar)
    part_num.append(item_code)
    actual_pn.append(full_prt)
prod_id_df= pd.DataFrame(
    {"part": part_num,
     "id": p_id,
     "Part Number": actual_pn
     })
    
prod_id_df

,part,id,Part Number
0,33-397,143,33-397-085
1,61-367,1289,61-367-030
2,45-478,1055,45-478-026
3,24-313,48,24-313-026
4,60-610,273,60-610-062
5,98-017,536,98-017
6,97-617,385,97-617
7,47-058,222,47-058-200
8,20-076,1462,20-076-100
9,24-383,54,24-383-026


In [ ]:
disc_items=['98-353-921','PM3616SM/MP','98-353-921','ABS4835SM-CG']
cust_item=['SV42-6201','45-270-216','45-253-026','45-271-026','45-383-026','97-464','45-549-026','45-622-251','45-270-026','97-909','24-391-026','24-190-085','45-304-026','97-942']
url_lst=['sv42-6201#',
        '45-272#/?color=white&cpu%20size%20option=Medium',
        '45-247#/?color=polished%20aluminum&cpu%20size%20option=Small',
        '45-273#/?color=polished%20aluminum&cpu%20size%20option=Medium',
        '45-353#/?capacity%20option=14–30%20lbs%20%2F%206.4–13.6%20kg&color=polished%20aluminum',
        'coiledextensioncordaccessorykit#/?power%20region=US%2FCA%2FMX',
        '45-509#/?color=polished%20aluminum',
        '45-621#/?color=white&worksurface%20option=Full%20worksurface',
        '45-272#/?color=polished%20aluminum&cpu%20size%20option=Medium',
        'svreplacementcoiledcordslacarts#/?power%20region=US%2FCA%2FMX',
        '24-390#/?capacity%20option=10–20%20lbs%20%2F%204.5–9.1%20kg&color=polished%20aluminum',
        '24-192#/?capacity%20option=25–50%20lbs%20%2F%2011.3–22.7%20kg&color=black',
        '45-296#/?capacity%20option=35–70%20lbs%20%2F%2015.9–31.8%20kg&color=polished%20aluminum',
        'svdcpowersystem#/?hardware%20configuration=for%20LCD%20%2B%20CPU']
cust_url= pd.DataFrame(
    {'part': cust_item,
    'url_tail': url_lst
    })
url_update = []
updt_prt = []
# for row in prod_id_df['part']:
#     if '98-353-921' in row:
#         prod_id_df['id'] = 'discontinued'
for row in range(len(cust_url)):
    url = "https://partner.ergotron.com/en-us/products/product-details/{}" .format((cust_url).loc[row, "url_tail"])
    # print(url)
    delay()
    r = requests.get(url, headers=headers)
    soup = BS(r.text, 'html.parser')
    try:
        target = soup.find("div", {"id": "dnn_ctr443_ContentPane"})
        target = str(target)
        tar = target.split("productId =",1)[1]
        tar = tar.split(",",1)[0]
        tar = re.sub("[^0-9]", "", tar)
    except:    
        tar = "URL nonmatch"
    url_update.append(tar)
cust_url['id']=url_update

cust_url

,part,url_tail,id
0,SV42-6201,sv42-6201#,881
1,45-270-216,45-272#/?color=white&cpu%20size%20option=Medium,198
2,45-253-026,45-247#/?color=polished%20aluminum&cpu%20size%...,186
3,45-271-026,45-273#/?color=polished%20aluminum&cpu%20size%...,199
4,45-383-026,45-353#/?capacity%20option=14–30%20lbs%20%2F%2...,206
5,97-464,coiledextensioncordaccessorykit#/?power%20regi...,554
6,45-549-026,45-509#/?color=polished%20aluminum,1095
7,45-622-251,45-621#/?color=white&worksurface%20option=Full...,1494
8,45-270-026,45-272#/?color=polished%20aluminum&cpu%20size%...,198
9,97-909,svreplacementcoiledcordslacarts#/?power%20regi...,556


**Backup and Reload Point**

 After successfully completeing the above opperations we should back up the variables to avoid repeated reruns of scraping opperations.

In [13]:
#backup dataframe variables
save("ergo_ps_backup", 'cust_url', 'pn2', 'prod_id_df', 'cust_item', 'disc_items')

In [3]:
#Load backup if needed
load("ergo_ps_backup")

In [4]:
#Correcting some of the missing IDs
prod_id_2 = []
prod_id_2 = prod_id_df.copy()
print(prod_id_2['id'].value_counts()['URL nonmatch'])
prod_id_2.set_index('part', inplace=True)
prod_id_2.update(cust_url.set_index('part'), overwrite=True)
prod_id_2.reset_index() 
print(prod_id_2['id'].value_counts()['URL nonmatch'])
prod_id_2



30
17


,id,Part Number
part,,
33-397,143,33-397-085
61-367,1289,61-367-030
45-478,1055,45-478-026
24-313,48,24-313-026
60-610,273,60-610-062
98-017,536,98-017
97-617,385,97-617
47-058,222,47-058-200
20-076,1462,20-076-100


In [6]:
#removing duplicates
print(len(prod_id_2))
prod_id_2 = prod_id_2.sort_values(['id']).drop_duplicates( keep='first')

print(len(prod_id_2))
prod_id_2

326
307


,id,Part Number
part,,
24-189,10,24-189-055
31-016,100,31-016-182
31-017,101,31-017-182
31-018,102,31-018-182
31-018,102,31-018-216
31-039,105,31-039-182
45-475,1054,45-475-026
45-475,1054,45-475-216
45-478,1055,45-478-026


In [7]:
prod_id_2['part']= prod_id_2.index
prod_id_2.reset_index(drop=True, inplace=True)
prod_id_2.head()

,id,Part Number,part
0,10,24-189-055,24-189
1,100,31-016-182,31-016
2,101,31-017-182,31-017
3,102,31-018-182,31-018
4,102,31-018-216,31-018


#### 5. Now we will take our product id list and add that to a new column in our dataframe and at the same time we will create a dataframe for those products for which we could not find a product ID. After looking over this list it mostly contains discontinued products.

In [30]:

pn3 =pn2.merge(right=prod_id_2[['Part Number','id']], left_on='Part Number', right_on='Part Number') 
print(pn3.shape)

# Check whether we have any missing ids:
redo = pn3[pn3.id.str.contains("URL nonmatch")]
pn3 = pn3[~pn3.id.str.contains("URL nonmatch")]
print(pn3.head())
print (redo)

In [10]:
#backup dataframe variables
save("ergo_ps_backup", 'cust_url', 'pn2', 'prod_id_df', 'cust_item', 'disc_items','pn3','ssp','part_name','pn3')

In [ ]:
#Load backup if needed
load("ergo_ps_backup")

#### 6. Determining whether items are discontinued or active and gathering SSP (suggested sale price) for each item.

With this site there is a field in the json file containing all the product info for each product that lists an "endDate" after which the product will be discontinued. We can read this date and determine if the parts are still active.

In [14]:

from datetime import datetime
import json
item7=pn3['id']
item8=pn3['Part Number']
headers = {'User-Agent': 'Mozilla/5.0'}
numbers = re.compile(r'\d+(?:\.\d+)?')
item_status = []
part_name = []
for part, num in zip(item7,item8):
    url = "https://partner.ergotron.com/DesktopModules/Ergotron/API/Client/GetProductDetails?culture=en-us&productId={}" .format(part)
    delay()
    r = requests.get(url, headers=headers).json()
    s = str(r)
    part_name.append(num)
    try:
        r_json = json.dumps(r['result'])
        r_json = json.loads(r_json)
        result_json = r_json[0]
        p1 = result_json["endDate"]
        try:
            p1_dt = datetime.strptime(p1,"%Y-%m-%dT%H:%M:%SZ")
            if p1_dt < datetime.today():
                item_status.append("EOL item")
            else:
                item_status.append('not EOL')
        except:
            item_status.append('no end date')
    except:
        item_status.append('invalid URL')


In [16]:
pn4 = pn3[["Product Name","Part Number","id"]]
pn4.insert(3, "Status", item_status, True)
print (pn4.shape)
pn4

(309, 4)


,Product Name,Part Number,id,Status
0,ERG-33-397-085,33-397-085,143,no end date
1,ERG-61-367-030,61-367-030,1289,no end date
2,ERG-45-478-026,45-478-026,1055,no end date
3,ERG-45-478-026,45-478-026,1055,no end date
4,ERG-24-313-026,24-313-026,48,EOL item
5,ERG-60-610-062,60-610-062,273,EOL item
6,ERG-60-610-062,60-610-062,273,EOL item
7,ERG-98-017,98-017,536,no end date
8,ERG-97-617,97-617,385,no end date
9,ERG-47-058-200,47-058-200,222,no end date


In [17]:
redo2 = pn4[pn4['Status'].str.contains('EOL item') == True]
pn4 = pn4[pn4['Status'].str.contains('EOL item') == False]
print(redo2.shape)
print(pn4.shape)

(34, 4)
(275, 4)


Next we can actually gather the SSP pricing in the same way by adding the id to the URL in order to access a json file for each part.

In [18]:
import json
item4=pn4['id']
item3=pn4['Part Number']
headers = {'User-Agent': 'Mozilla/5.0'}
numbers = re.compile(r'\d+(?:\.\d+)?')
ssp = []
part_name = []
for part, num in zip(item4,item3):
    url = "https://partner.ergotron.com/DesktopModules/Ergotron/API/Client/GetProductDetails?culture=en-us&productId={}" .format(part)
    delay()
    r = requests.get(url, headers=headers).json()
    s = str(r)
    try:
        txt1 = s.split("'partNumber': '{}'".format(num),1)[1]
        txt2 = txt1.split("msrPrice",1)[1]
        txt3 = txt2.split(',',1)[0]
        val1 = numbers.findall(txt3)
        ssp.append(val1)
    except:
        try:
            txt1 = s.split("'partNumber': '{} '".format(num),1)[1]
            txt2 = txt1.split("msrPrice",1)[1]
            txt3 = txt2.split(',',1)[0]
            val1 = numbers.findall(txt3)
            ssp.append(val1)
        except:
            try:
                r_json = json.dumps(r['result'])
                r_json = json.loads(r_json)
                result_json = r_json[0]
                p1 = result_json['listPrice']
                val1 = numbers.findall(str(p1))
                ssp.append(val1)
            except:
                ssp.append("noprice")

    try:
        txt4 = txt1.split("'partName': ",1)[1]
        name1 = txt4.split(", 'msrPrice'",1)[0]
        part_name.append(name1)
    except:
        part_name.append("noname")
print(ssp)
print(part_name)

[['590.0'], ['1398.0'], ['450.0'], ['450.0'], ['25.0'], ['114.0'], ['115.0'], ['397.0'], ['150.0'], ['2702.0'], ['1069.0'], ['1069.0'], ['1069.0'], ['70.0'], ['86.0'], ['40.0'], ['47.0'], ['39.0'], ['517.0'], ['212.0'], ['1201.0'], ['1078.0'], ['1960.0'], ['55.0'], ['706.0'], ['5714.0'], ['106.0'], ['106.0'], ['559.0'], ['559.0'], ['119.0'], ['1028.0'], ['780.0'], ['175.0'], ['1016.0'], ['437.0'], ['271.0'], ['164.0'], [], ['90.0'], ['833.0'], ['40.0'], ['450.0'], ['55.0'], ['1272.0'], ['174.0'], ['1398.0'], ['1136.0'], ['173.0'], ['598.0'], ['121.0'], ['121.0'], ['204.0'], ['424.0'], ['129.0'], ['247.0'], ['133.0'], ['104.0'], ['833.0'], ['842.0'], ['1018.0'], ['1349.0'], ['72.0'], ['55.0'], ['1883.0'], ['710.0'], ['1847.0'], ['39.0'], ['39.0'], ['274.0'], ['59.0'], ['59.0'], ['79.0'], ['1012.0'], ['304.0'], ['77.0'], ['5881.0'], ['144.0'], ['110.0'], ['93.0'], ['279.0'], ['118.0'], ['273.0'], ['228.0'], ['151.0'], ['165.0'], ['205.0'], ['428.0'], ['387.0'], ['1199.0'], ['471.0'], ['7

#### 7. Next let's create new columns in our main dataframe for these new values we've gathered.

In [20]:
pn5 = pn4[["Product Name","Part Number","id","Status"]]
pn5.insert(4, "SSP", ssp, True)
pn5.insert(5, "Description", part_name, True)
print (pn5)

        Product Name  Part Number    id       Status       SSP  \
0     ERG-33-397-085   33-397-085   143  no end date   [590.0]   
1     ERG-61-367-030   61-367-030  1289  no end date  [1398.0]   
2     ERG-45-478-026   45-478-026  1055  no end date   [450.0]   
3     ERG-45-478-026   45-478-026  1055  no end date   [450.0]   
7         ERG-98-017       98-017   536  no end date    [25.0]   
8         ERG-97-617       97-617   385  no end date   [114.0]   
9     ERG-47-058-200   47-058-200   222  no end date   [115.0]   
10    ERG-20-076-100   20-076-100  1462  no end date   [397.0]   
14    ERG-97-487-800   97-487-800   331  no end date   [150.0]   
15   ERG-SV43-1120-0  SV43-1120-0   886  no end date  [2702.0]   
16    ERG-C50-1100-0   C50-1100-0  1250  no end date  [1069.0]   
17    ERG-C50-1100-0   C50-1100-0  1250  no end date  [1069.0]   
18    ERG-C50-1100-0   C50-1100-0  1250  no end date  [1069.0]   
21    ERG-47-092-800   47-092-800   223  no end date    [70.0]   
22    ERG-

In [25]:
#check if we have any unexpected values in ssp
# print(pn5[pn5['SSP']=='noprice'])
# print(pn5[pn5['SSP']=='[]'])
# pn5[pn5['SSP'].str.contains('') == True]


,Product Name,Part Number,id,Status,SSP,Description


#### 8. Clean the data so that text and number formating is correct.

In [26]:
pn5['SSP'] = pn5['SSP'].astype(str)
pn5['SSP']=pn5['SSP'].str.replace('[','', regex=True).str.replace(']','', regex=True)
pn5['SSP']=pn5['SSP'].str.replace("'",'', regex=True).str.replace("'",'', regex=True)
pn5['Description']=pn5['Description'].str.replace(",",'', regex=True)
pn5['SSP']=pn5['SSP'].replace('', np.nan)
pn5['SSP'].fillna(0, inplace=True)
pn5=pn5.astype({"SSP":"float"})
pn5

,Product Name,Part Number,id,Status,SSP,Description
0,ERG-33-397-085,33-397-085,143,no end date,590.0,"'WorkFit-T, Standing Desk Workstation (black w..."
1,ERG-61-367-030,61-367-030,1289,no end date,1398.0,'CareFit™ Enclosure (white)'
2,ERG-45-478-026,45-478-026,1055,no end date,450.0,'HX Wall Monitor Arm (polished aluminum)'
3,ERG-45-478-026,45-478-026,1055,no end date,450.0,'HX Wall Monitor Arm (polished aluminum)'
7,ERG-98-017,98-017,536,no end date,25.0,'LX Arm Grommet Mount'
8,ERG-97-617,97-617,385,no end date,114.0,'WorkFit Conversion Kit: Dual to LCD & Laptop'
9,ERG-47-058-200,47-058-200,222,no end date,115.0,'DS100 Clamping Single Pivot'
10,ERG-20-076-100,20-076-100,1462,no end date,397.0,"'Command Post - 3"" x 3"" x 80"" Post'"
14,ERG-97-487-800,97-487-800,331,no end date,150.0,'Vertical Lift Auto-Retracting Keyboard Kit'
15,ERG-SV43-1120-0,SV43-1120-0,886,no end date,2702.0,"'StyleView® Laptop Cart, 2 Drawers (1x2)'"


In [27]:
#drop rows with ssp == 0
print (pn5.shape)
pn5 = pn5[pn5['SSP'] != 0]
print (pn5.shape)

(275, 6)
(268, 6)


In [31]:
#backup dataframe variables
save("ergo_ps_backup", 'cust_url', 'pn2', 'prod_id_df', 'cust_item', 'disc_items','pn3','ssp','part_name','pn3','pn4','pn5','redo','redo2')

In [4]:
#Load backup if needed
load("ergo_ps_backup")

#### Check the amount of price change to look for unexpected changes

In [5]:
# Checking for large changes against exisitng prices

pn2_dd = pn2.drop_duplicates(subset='Product Name', keep ='first')
pn5_dd = pn5.drop_duplicates(subset='Product Name', keep = 'first')
pn5_dd = pn5_dd.merge(pn2_dd[['Product Name','Price']], how = 'left', left_on='Product Name',right_on='Product Name')
pn5_dd

,Product Name,Part Number,id,Status,SSP,Description,Price
0,ERG-33-397-085,33-397-085,143,no end date,590.0,"'WorkFit-T, Standing Desk Workstation (black w...",$449.00
1,ERG-61-367-030,61-367-030,1289,no end date,1398.0,'CareFit™ Enclosure (white)',"$1,233.00"
2,ERG-45-478-026,45-478-026,1055,no end date,450.0,'HX Wall Monitor Arm (polished aluminum)',$325.00
3,ERG-98-017,98-017,536,no end date,25.0,'LX Arm Grommet Mount',$17.00
4,ERG-97-617,97-617,385,no end date,114.0,'WorkFit Conversion Kit: Dual to LCD & Laptop',$62.00
5,ERG-47-058-200,47-058-200,222,no end date,115.0,'DS100 Clamping Single Pivot',$93.00
6,ERG-20-076-100,20-076-100,1462,no end date,397.0,"'Command Post - 3"" x 3"" x 80"" Post'",$220.00
7,ERG-97-487-800,97-487-800,331,no end date,150.0,'Vertical Lift Auto-Retracting Keyboard Kit',$131.00
8,ERG-SV43-1120-0,SV43-1120-0,886,no end date,2702.0,"'StyleView® Laptop Cart, 2 Drawers (1x2)'","$2,216.00"
9,ERG-C50-1100-0,C50-1100-0,1250,no end date,1069.0,'CareFit™ Slim Laptop Cart',$800.00


In [6]:
pn5_dd['Price'] = pn5_dd['Price'].astype(str)
pn5_dd['Price']=pn5_dd['Price'].str.replace('$','', regex=True).str.replace(',','', regex=True)

pn5_dd.head()

,Product Name,Part Number,id,Status,SSP,Description,Price
0,ERG-33-397-085,33-397-085,143,no end date,590.0,"'WorkFit-T, Standing Desk Workstation (black w...",449.00
1,ERG-61-367-030,61-367-030,1289,no end date,1398.0,'CareFit™ Enclosure (white)',1233.00
2,ERG-45-478-026,45-478-026,1055,no end date,450.0,'HX Wall Monitor Arm (polished aluminum)',325.00
3,ERG-98-017,98-017,536,no end date,25.0,'LX Arm Grommet Mount',17.00
4,ERG-97-617,97-617,385,no end date,114.0,'WorkFit Conversion Kit: Dual to LCD & Laptop',62.00


In [11]:
#adding a column for amount of change in price
pn5_dd['Percent Change']=1-(pn5_dd['Price'].astype(float)/pn5_dd['SSP'].astype(float))
pn5_dd.head()

,Product Name,Part Number,id,Status,SSP,Description,Price,Percent Change
0,ERG-33-397-085,33-397-085,143,no end date,590.0,"'WorkFit-T, Standing Desk Workstation (black w...",449.00,0.238983
1,ERG-61-367-030,61-367-030,1289,no end date,1398.0,'CareFit™ Enclosure (white)',1233.00,0.118026
2,ERG-45-478-026,45-478-026,1055,no end date,450.0,'HX Wall Monitor Arm (polished aluminum)',325.00,0.277778
3,ERG-98-017,98-017,536,no end date,25.0,'LX Arm Grommet Mount',17.00,0.320000
4,ERG-97-617,97-617,385,no end date,114.0,'WorkFit Conversion Kit: Dual to LCD & Laptop',62.00,0.456140


In [14]:
#Create a new df for unusual price changes to double check before changes made in system
flag_price = pn5_dd.loc[(pn5_dd['Percent Change']>.4)|(pn5_dd['Percent Change']<0)]
flag_price

,Product Name,Part Number,id,Status,SSP,Description,Price,Percent Change
4,ERG-97-617,97-617,385,no end date,114.0,'WorkFit Conversion Kit: Dual to LCD & Laptop',62.00,0.456140
6,ERG-20-076-100,20-076-100,1462,no end date,397.0,"'Command Post - 3"" x 3"" x 80"" Post'",220.00,0.445844
25,ERG-687BK,687BK,1177,no end date,119.0,'Swing-Out Mouse Shelf',52.00,0.563025
57,ERG-YES40-TAB-1,YES40-TAB-1,1293,no end date,1883.0,'YES40 Charging Cart',2665.00,-0.415295
59,ERG-DM12-1006-1,DM12-1006-1,582,no end date,1847.0,"'Zip12 Charging Wall Cabinet, US/CA/MX'",777.00,0.579318
69,ERG-98-150-055,98-150-055,1119,no end date,110.0,'Keyboard Tray with Debris Barrier Upgrade Kit...,125.00,-0.136364
87,ERG-97-565,97-565,354,no end date,54.0,'SV40 Basket and Handle Kit',70.00,-0.296296
90,ERG-97-780-194,97-780-194,436,no end date,69.0,'SV Scanner Shelf',30.00,0.565217
91,ERG-97-848,97-848,462,no end date,41.0,"'SV Replacement Drawer Kit, Single'",55.00,-0.341463
92,ERG-24-811-F13,24-811-F13,1306,no end date,499.0,'Mobile Desk',655.00,-0.312625


In [15]:
#One item had an incorrect price. Need to check this later.
pn5_dd.loc[150,'SSP'] = float(284.0)
pn5_dd.loc[150]

Product Name                               ERG-ZDDCGCG
Part Number                                    ZDDCGCG
id                                                 982
Status                                     no end date
SSP                                              284.0
Description       'SV43/44 Supplemental Single Drawer'
Price                                          209.00 
Percent Change                                0.402857
Name: 150, dtype: object

In [32]:
# del crm_1

#### Add CRM system unique IDs
In order to properly import the updated information into the CRM system we needed to match the parts with the unique ID numbers generated by the CRM. In the future this could be done on the initial import.

In [7]:

crm_1 = pd.read_csv('crm_1.csv',  thousands=",")
crm_1.rename(columns={"ID": "crm_ID"}, inplace=True)
crm_1.drop(columns="Cost", inplace=True)
crm_1.head()

,Product Name,crm_ID,Description,Part Number,Price
0,ERG-33-397-085,1001cee2-12d6-dabd-0765-54aec2451f45,"WorkFit-T, Sit-Stand Desktop Workstation (black)",33-397-085,$590.00
1,ERG-61-367-030,11bd2dfd-f754-a5d2-c45d-5e5eaf825641,CareFit Enclosure (snow white) Wall-mount comp...,61-367-030,"$1,233.00"
2,ERG-45-478-026,11ca3cd7-405c-38ca-42c4-58d43d63d597,HX Wall Mount Monitor Arm (Polished Aluminum),45-478-026,$325.00
3,ERG-24-313-026,120c6e18-bd67-4b6b-ff5a-53ee6dd096f9,"WorkFit-A, Single LD",24-313-026,$409.00
4,ERG-60-610-062,13196af5-5189-69ef-d247-528634d65620,"StyleView Sit-Stand Enclosure (bright white), ...",60-610-062,"$1,112.00"


In [35]:
pn5_dd = pn5_dd.merge(crm_1[['Product Name','crm_ID']], how = 'left', left_on='Product Name',right_on='Product Name')
pn5_dd.head()

,Product Name,Part Number,id,Status,SSP,Description,Price,Percent Change,crm_ID
0,ERG-33-397-085,33-397-085,143,no end date,590.0,'WorkFit-T Standing Desk Workstation (black wi...,449.00,0.238983,1001cee2-12d6-dabd-0765-54aec2451f45
1,ERG-61-367-030,61-367-030,1289,no end date,1398.0,'CareFit™ Enclosure (white)',1233.00,0.118026,11bd2dfd-f754-a5d2-c45d-5e5eaf825641
2,ERG-45-478-026,45-478-026,1055,no end date,450.0,'HX Wall Monitor Arm (polished aluminum)',325.00,0.277778,11ca3cd7-405c-38ca-42c4-58d43d63d597
3,ERG-45-478-026,45-478-026,1055,no end date,450.0,'HX Wall Monitor Arm (polished aluminum)',325.00,0.277778,6f07ea7f-61f9-2cbd-f832-5d3f48380849
4,ERG-98-017,98-017,536,no end date,25.0,'LX Arm Grommet Mount',17.00,0.320000,1381e37e-fc5d-9bff-0a59-5d5bff0bc399


#### CSV File Output

In [ ]:
redo.to_csv(r'./inv_items.csv', index = False)
redo2.to_csv(r'./disc_items.csv', index = False)

In [36]:
pn5_dd.to_csv(r'./erg_web_price.csv', index = False)


## Backup and Reload Point

In [7]:
save("ergo_ps_backup", 'cust_url', 'pn2', 'prod_id_df', 'cust_item', 'disc_items','pn3','ssp','part_name','pn3','pn4','pn5','redo','redo2','pn5_dd')

In [3]:
#Load backup if needed
load("ergo_ps_backup")